<a href="https://colab.research.google.com/github/pedrogengo/DLforNLP/blob/main/Aula_2_Exerc%C3%ADcio_Pedro_Gengo_BoW_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 2: Análise de Sentimentos usando Bag of Words e TF-IDF
Nome: Pedro Gabriel Gengo Lourenço

## Enunciado

- Treinar um classificador binário na tarefa de análise de sentimentos usando dataset IMDB.

- Experimentar e reportar a acurácia usando 3 diferentes tipos de features como entrada:
    1) Bag-of-words booleano
    2) Bag-of-words com contagem das palavras (histograma das palavras)
    3) TF-IDF


As funções de tokenização e conversão de tokens para features devem ser implementas usando apenas o numpy ou outros pacotes nativos do python. Não é permitido usar as funções prontas (ex: do scikit-learn) para se obter as features de entrada.


O scikit-learn pode ser usado apenas para treinar e avaliar o classificador (ex: SVM).

Neste notebook iremos treinar um modelo para fazer análise de sentimento usando o dataset IMDB.

# Preparando Dados

Primeiro, fazemos download do dataset:

In [ ]:
!wget -nc http://files.fast.ai/data/examples/imdb_sample.tgz
!tar -xzf imdb_sample.tgz

File ‘imdb_sample.tgz’ already there; not retrieving.



Carregamos o dataset .csv usando o pandas:

In [ ]:
import pandas as pd
df = pd.read_csv('imdb_sample/texts.csv')
print(df.shape)
df.head()

(1000, 3)


,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


Iremos agora dividir o dataset em conjuntos de treino e teste:

In [ ]:
treino = df[df['is_valid'] == False]
valid = df[df['is_valid'] == True]

print('treino.shape:', treino.shape)
print('valid.shape:', valid.shape)

treino.shape: (800, 3)
valid.shape: (200, 3)


E iremos dividir estes dois conjuntos em entrada (X) e saída desejada (Y, ground-truth) do modelo:

In [ ]:
X_treino = treino['text']
Y_treino = treino['label']
X_valid = valid['text']
Y_valid = valid['label']

print('X_treino.head():', X_treino.head())
print('Y_treino.head():', Y_treino.head())

X_treino.head(): 0    Un-bleeping-believable! Meg Ryan doesn't even ...
1    This is a extremely well-made film. The acting...
2    Every once in a long while a movie will come a...
3    Name just says it all. I watched this movie wi...
4    This movie succeeds at being one of the most u...
Name: text, dtype: object
Y_treino.head(): 0    negative
1    positive
2    negative
3    positive
4    negative
Name: label, dtype: object


Ainda falta converter as strings "positive" e "negative" do ground-truth para valores booleanos:

In [ ]:
mapeamento = {'positive': True, 'negative': False}
Y_treino_bool = Y_treino.map(mapeamento)
Y_valid_bool = Y_valid.map(mapeamento)
print(Y_treino_bool.head())

0    False
1     True
2    False
3     True
4    False
Name: label, dtype: bool


## Processamento dos textos

Nessa etapa, como descrito no enunciado do exercício, iremos realizar três tipos de processamento:

1. BoW booleano
2. BoW com base na frequência
3. TF-IDF

É muito importante ressaltar a importância de aplicar a "aprender" transformação apenas no treino, ou seja, utilizar apenas o vocabulário do treino, e utilizar o que foi encontrado para o teste/validação.

Para isso, usarei a estrutura de fit/transform bastante conhecida da biblioteca sklearn. Começarei, então, criando uma classe abstrata que será herdada na criação das outras.

In [ ]:
from abc import ABC, abstractmethod
 
class Transformer(ABC):
 
  @abstractmethod
  def fit(self):
      pass
    
  @abstractmethod
  def transform(self):
      pass

## BoW (booleano e com frequência)

In [ ]:
from collections import Counter
import numpy as np

class BagOfWords(Transformer):
  '''
  Essa classe realiza a transformacao de uma lista de palavras
  para uma lista de inteiros.

  Attrs:
    boolean(bool): Flag que define se o vetor gerado sera com base na
      frequencia (contagem) ou com base na ocorrencia ou nao (bool) de
      uma palavra do vocabulario.
    max_size(int): Define o tamanho maximo do vocabulario. Caso usado com
      use_unknown = True, o vocabulario tera o tamanho de max_size + 1.
    stopwords(list): Define a lista de palavras que serao desconsideras na
      geracao do vocabulario.
    use_unknown(bool): Flag que define o uso ou nao de um elemento para
      palavras que nao existem no vocabulario.
  '''

  def __init__(self, boolean=False, max_size=None, stopwords = [], use_unknown=False):
    self.max_size = max_size
    self.boolean = boolean
    self.stopwords = stopwords
    self.use_unknown = use_unknown

  def _create_vocab(self, tokenized_texts):
    '''
    Cria o vocabulario que sera utilizado na transformacao do vetores
    de palavras para vetores de inteiros.

    Args:
      tokenized_texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    
    Return:
      vocab(dict): Dicionario onde as chaves sao as palavras do vocabulario
        e os valores representam o indice da palavra no vetor a ser gerado.
    '''
    counter = Counter()
    for text in tokenized_texts:
      counter.update(text)
    for stop_word in self.stopwords:
      if stop_word in counter.keys():
        del counter[stop_word]
    vocab = {element[0]: index for index, element in enumerate(counter.most_common(self.max_size))}
    if self.use_unknown:
      vocab['unknown'] = len(vocab)
    return vocab
  
  def fit(self, texts):
    '''
    Metodo que cria os argumentos que serao utilizados nas
    transformacoes posteriores. Esse metodo so deve ser utilizado 
    sobre o conjunto de treino.

    Args:
      texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    '''
    vocab = self._create_vocab(texts)
    self.vocabulary = vocab

  def transform(self, texts):
    '''
    Realiza a transformacao de uma lista de tokens para uma
    lista de inteiros com base no vocabulario criado na etapa
    de fit.

    Args:
      texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    
    Return:
      bow_texts(np.array): Array contendo os vetores de tokens
        transformados para vetores de inteiros de tamanho fixo.
    '''
    transformed_texts = []
    if self.use_unknown:
      unknown = self.vocabulary.get('unknown')

    for i, text in enumerate(texts):
      bow_text = np.zeros(len(self.vocabulary))
      counter = Counter(text)

      if self.use_unknown:
        index = [self.vocabulary.get(key, unknown) for key in counter.keys()]
      else:
        index = [self.vocabulary[key] for key in counter.keys() if key in self.vocabulary.keys()]

      if self.boolean:
        bow_text[index] = 1
      else:
        values = [value for key, value in counter.items() if self.use_unknown or key in self.vocabulary.keys()]
        bow_text[index] = values

      transformed_texts.append(bow_text)

    return np.vstack(transformed_texts)

In [ ]:
## Vocabulario deve ser: {'a': 0, 'texttinho': 1, 'testando': 2, 'b': 3, 'c': 4, 'unknown': 5}
texts_test = [['text', 'texttinho', 'texttinho', 'testando'], ['a', 'b', 'a', 'a', 'c']]

## Testando BoW com frequencia
bow = BagOfWords(boolean=False, stopwords=['text'], use_unknown=True)
bow.fit(texts_test)
assert np.all(bow.transform(texts_test) == np.array([[0., 2., 1., 0., 0., 1.], [3., 0., 0., 1., 1., 0.]]))

## Testando BoW booleano
bow = BagOfWords(boolean=True, stopwords=['text'], use_unknown=False)
bow.fit(texts_test)
assert np.all(bow.transform(texts_test) == np.array([[0., 1., 1., 0., 0.], [1., 0., 0., 1., 1.]]))

## TF-IDF

$$\text{TF-IDF}(t, d, C) = tf(t, d) * idf(t, C)$$

Abrindo as funções definidas na equação principal:
- $tf(t, d) = \text{numero de vezes que o termo t aparece no documento d}$
- $idf(t, C) = \log{\frac{C}{n_t}}$ 

Onde: 

- $\text{t: token ou termo;}$
- $\text{d: documento(frase, enunciado, etc);}$
- $\text{C: Corpus (conjunto de documentos).}$
- $n_t\text{: numero de documentos onde o token t aparece.}$


 



In [ ]:
from collections import Counter
import numpy as np

class TfIdf(Transformer):
  '''
  Essa classe realiza a transformacao de uma lista de palavras
  para uma lista de inteiros utilizando TFIDF.

  Attrs:
    max_size(int): Define o tamanho maximo do vocabulario. Caso usado com
      use_unknown = True, o vocabulario tera o tamanho de max_size + 1.
    stopwords(list): Define a lista de palavras que serao desconsideras na
      geracao do vocabulario.
  '''

  def __init__(self, max_size=None, stopwords = []):
    self.max_size = max_size
    self.stopwords = stopwords

  def _count_tokens_in_doc(self, tokenized_texts):
    '''
    Realiza a contagem de em quantos documentos uma mesma
    palavra aparece, desconsiderando as stopwords.

    Args:
      tokenized_texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    
    Return:
      counter(collections.Counter): Objeto da classe Counter com todos
        os elementos do conjunto de treino.
    '''
    counter = Counter()
    for text in tokenized_texts:
      counter.update(set(text))
    for stop_word in self.stopwords:
      if stop_word in counter.keys():
        del counter[stop_word]
    return counter
  
  def _create_idf(self, counter):
    '''
    Cria o vetor de idf para cada um dos tokens do conjunto de treino.

    Args:
      counter(collections.Counter): Objeto da classe Counter com todos
        os elementos do conjunto de treino.
    
    Return:
      idf(np.array): Array contendo o valor de idf para cada um dos tokens
        do conjunto de treino.
    '''
    idf = [self.len_corpus/count for token, count in counter.most_common(self.max_size)]
    return np.log(idf)

  def _create_vocab(self, counter):
    '''
    Cria o vocabulario que sera utilizado na transformacao do vetores
    de palavras para vetores de inteiros.

    Args:
      counter(collections.Counter): Objeto da classe Counter com todos
        os elementos do conjunto de treino.
    
    Return:
      vocab(dict): Dicionario onde as chaves sao as palavras do vocabulario
        e os valores representam o indice da palavra no vetor a ser gerado.
    '''
    vocab = {element[0]: index for index, element in enumerate(counter.most_common(self.max_size))}
    return vocab

  def fit(self, texts):
    '''
    Metodo que cria os argumentos que serao utilizados nas
    transformacoes posteriores. Esse metodo so deve ser utilizado 
    sobre o conjunto de treino.

    Args:
      texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    '''
    self.len_corpus = len(texts)
    counter = self._count_tokens_in_doc(texts)

    self.vocabulary = self._create_vocab(counter)
    self.idf = self._create_idf(counter)

  def transform(self, texts):
    '''
    Realiza a transformacao de uma lista de tokens para uma
    lista de inteiros com base no vocabulario criado na etapa
    de fit.

    Args:
      texts(list): Lista de textos ja tokenizados, ou seja,
        uma lista onde cada elemento e um token.
    
    Return:
      tfidf_texts(np.array): Array contendo os vetores de tokens
        transformados para vetores de inteiros de tamanho fixo.
    '''
    transformed_texts = []

    for i, text in enumerate(texts):
      bow_text = np.zeros(len(self.vocabulary))
      counter = Counter(text)

      index = []
      values = []
      for key, value in counter.items():
        if key in self.vocabulary.keys():
          index.append(self.vocabulary[key])
          values.append(value)

      bow_text[index] = values

      transformed_texts.append(bow_text * self.idf)

    return np.vstack(transformed_texts)

In [ ]:
## Vocabulario: {'t1': 0, 't2': 1, 't3': 2, 't4': 3}

## Validando o TFIDF
texts = [['t1', 't2', 't3', 't2', 't1'], ['t2', 't1'], ['t4', 't1']]
tfidf = TfIdf()
tfidf.fit(texts)
assert np.all(tfidf.transform(texts) - np.array([[0., 2 * np.log(3/2), np.log(3), 0.], [0., np.log(3/2), 0., 0.] , [0., 0., 0., np.log(3)]]) < 0.001)

## Treinando um classificador binário

Antes de treinar o classificador, precisamos garantir que nossos textos de entradas estejam vetorizados, ou seja, tenhamos aplicado algum método de conversão de token para features, que são as classes que definimos acima.
Contudo, para que utilizemos as classes, necessitamos tokenizar nossos textos antes. Usei a estratégia de realizar a tokenização fora das classes pois assim, posso reutilizar os textos já tokenizados para os três experimentos que irei efetuar.

### Tokenização

Irei aplicar uma tokenização simples, onde irei remover a pontuação do texto e irei dividí-lo por palavras, ou seja, meus tokens serão as palavras que compõe a avaliação do filme.

In [ ]:
from re import findall

def tokenizer(texts):
  tokenized_texts = []
  for text in texts:
    tokens = findall(r'\w+|[^?\-!.,:;"\'/><\s]', text)
    tokenized = [token.lower() for token in tokens]
    tokenized_texts.append(tokenized)
  return tokenized_texts

In [ ]:
tokenized_X_treino = tokenizer(X_treino.values)
tokenized_X_valid = tokenizer(X_valid.values)

### Acurácia

Os resultados dos experimentados serão dados em função da acurácia, que representa o quanto acertamos do total. Para isso, necessitei escrever a função que irá calcular ela.

In [ ]:
def accuracy(y_true, y_pred):
  return np.sum(y_true == y_pred) / y_true.shape[0]

### Experimentos

A função abaixo foi definida para facilitar a execução dos experimentos utilizando os diferentes tipos de conversão de tokens para features.

In [ ]:
def run(tokenized_train, tokenized_valid, target_train, target_valid, vectorizer, model):
  print(f'Aplicando {vectorizer.__class__.__name__}')
  vectorizer.fit(tokenized_train)
  vectorized_texts = vectorizer.transform(tokenized_train)
  print(f'O shape dos dados de treinos vetorizados é: {vectorized_texts.shape}')
  print(f'Treinando {model.__class__.__name__}')
  model.fit(vectorized_texts, target_train)

  print(f'Utilizando os dados de validação')
  vectorized_valid = vectorizer.transform(tokenized_valid)
  y_predicted = model.predict(vectorized_valid)

  acc = round(accuracy(target_valid, y_predicted), 4) * 100
  print('*' * 40)
  print(f'Acurácia de {acc}% utilizando {vectorizer.__class__.__name__}')
  print('*' * 40)

#### Experimento 1: BoW booleano

In [ ]:
from sklearn.svm import SVC

In [ ]:
bow_bool = BagOfWords(boolean=True, max_size=3000)
model = SVC(C=10.)
run(tokenized_X_treino, tokenized_X_valid, Y_treino_bool, Y_valid_bool, bow_bool, model)

Aplicando BagOfWords
O shape dos dados de treinos vetorizados é: (800, 3000)
Treinando SVC
Utilizando os dados de validação
****************************************
Acurácia de 82.5% utilizando BagOfWords
****************************************


#### Experimento 2: BoW frequencia

In [ ]:
bow_freq = BagOfWords(max_size=3000)
model = SVC(C=10.)
run(tokenized_X_treino, tokenized_X_valid, Y_treino_bool, Y_valid_bool, bow_freq, model)

Aplicando BagOfWords
O shape dos dados de treinos vetorizados é: (800, 3000)
Treinando SVC
Utilizando os dados de validação
****************************************
Acurácia de 77.0% utilizando BagOfWords
****************************************


#### Experimento 3: TF-IDF

In [ ]:
tfidf = TfIdf(max_size=3000)
model = SVC(C=10.)
run(tokenized_X_treino, tokenized_X_valid, Y_treino_bool, Y_valid_bool, tfidf, model)

Aplicando TfIdf
O shape dos dados de treinos vetorizados é: (800, 3000)
Treinando SVC
Utilizando os dados de validação
****************************************
Acurácia de 83.5% utilizando TfIdf
****************************************


### Extra: Removendo stopwords

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_english = list(stopwords.words('english'))
sw_english[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

#### Experimento 1: BoW booleano

In [ ]:
bow_bool = BagOfWords(boolean=True, max_size=3000, stopwords=sw_english)
model = SVC(C=10.)
run(tokenized_X_treino, tokenized_X_valid, Y_treino_bool, Y_valid_bool, bow_bool, model)

Aplicando BagOfWords
O shape dos dados de treinos vetorizados é: (800, 3000)
Treinando SVC
Utilizando os dados de validação
****************************************
Acurácia de 81.0% utilizando BagOfWords
****************************************


#### Experimento 2: BoW frequencia

In [ ]:
bow_freq = BagOfWords(max_size=3000, stopwords=sw_english)
model = SVC(C=10.)
run(tokenized_X_treino, tokenized_X_valid, Y_treino_bool, Y_valid_bool, bow_freq, model)

Aplicando BagOfWords
O shape dos dados de treinos vetorizados é: (800, 3000)
Treinando SVC
Utilizando os dados de validação
****************************************
Acurácia de 83.5% utilizando BagOfWords
****************************************


#### Experimento 3: TF-IDF

In [ ]:
tfidf = TfIdf(max_size=3000, stopwords=sw_english)
model = SVC(C=10.)
run(tokenized_X_treino, tokenized_X_valid, Y_treino_bool, Y_valid_bool, tfidf, model)

Aplicando TfIdf
O shape dos dados de treinos vetorizados é: (800, 3000)
Treinando SVC
Utilizando os dados de validação
****************************************
Acurácia de 82.5% utilizando TfIdf
****************************************
